In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
data = 'https://raw.githubusercontent.com/alexeygrigorev/mlbookcamp-code/master/chapter-03-churn-prediction/WA_Fn-UseC_-Telco-Customer-Churn.csv'

In [3]:
!wget $data -O data-week-3.csv 

--2022-06-20 15:47:47--  https://raw.githubusercontent.com/alexeygrigorev/mlbookcamp-code/master/chapter-03-churn-prediction/WA_Fn-UseC_-Telco-Customer-Churn.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 977501 (955K) [text/plain]
Saving to: ‘data-week-3.csv’

data-week-3.csv     100%[===================>] 954.59K  --.-KB/s    in 0.05s   

2022-06-20 15:47:47 (20.5 MB/s) - ‘data-week-3.csv’ saved [977501/977501]



In [8]:
df = pd.read_csv('data-week-3.csv')
df.head()

,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,...,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,7590-VHVEG,Female,0,Yes,No,1,No,No phone service,DSL,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,29.85,29.85,No
1,5575-GNVDE,Male,0,No,No,34,Yes,No,DSL,Yes,...,Yes,No,No,No,One year,No,Mailed check,56.95,1889.5,No
2,3668-QPYBK,Male,0,No,No,2,Yes,No,DSL,Yes,...,No,No,No,No,Month-to-month,Yes,Mailed check,53.85,108.15,Yes
3,7795-CFOCW,Male,0,No,No,45,No,No phone service,DSL,Yes,...,Yes,Yes,No,No,One year,No,Bank transfer (automatic),42.30,1840.75,No
4,9237-HQITU,Female,0,No,No,2,Yes,No,Fiber optic,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,70.70,151.65,Yes


In [17]:
df.columns = df.columns.str.lower().str.replace(' ', '_')

categroical_columns = list(df.dtypes[df.dtypes == 'object'].index)

for c in categroical_columns:
  df[c] = df[c].str.lower().str.replace(' ', '_')

In [19]:
df.head().T

,0,1,2,3,4
customerid,7590-vhveg,5575-gnvde,3668-qpybk,7795-cfocw,9237-hqitu
gender,female,male,male,male,female
seniorcitizen,0,0,0,0,0
partner,yes,no,no,no,no
dependents,no,no,no,no,no
tenure,1,34,2,45,2
phoneservice,no,yes,yes,no,yes
multiplelines,no_phone_service,no,no,no_phone_service,no
internetservice,dsl,dsl,dsl,dsl,fiber_optic
onlinesecurity,no,yes,yes,yes,no


In [20]:
df.dtypes

customerid           object
gender               object
seniorcitizen         int64
partner              object
dependents           object
tenure                int64
phoneservice         object
multiplelines        object
internetservice      object
onlinesecurity       object
onlinebackup         object
deviceprotection     object
techsupport          object
streamingtv          object
streamingmovies      object
contract             object
paperlessbilling     object
paymentmethod        object
monthlycharges      float64
totalcharges         object
churn                object
dtype: object

In [36]:
df.totalcharges = pd.to_numeric(df.totalcharges, errors = 'coerce')
df.totalcharges

0         29.85
1       1889.50
2        108.15
3       1840.75
4        151.65
         ...   
7038    1990.50
7039    7362.90
7040     346.45
7041     306.60
7042    6844.50
Name: totalcharges, Length: 7043, dtype: float64

In [42]:
df.totalcharges = df.totalcharges.fillna(0)

In [46]:
df.churn = (df.churn == 'yes').astype(int)

In [47]:
df.churn

0       0
1       0
2       1
3       0
4       1
       ..
7038    0
7039    0
7040    0
7041    1
7042    0
Name: churn, Length: 7043, dtype: int64

In [49]:
from sklearn.model_selection import train_test_split

full_train , test = train_test_split(df, test_size = 0.2, random_state = 1)

In [52]:
train , val  = train_test_split(full_train, test_size = 0.25, random_state= 1)

In [65]:
len(train), len(val), len(test)

(4225, 1409, 1409)

In [53]:
full_train = full_train.reset_index(drop = True)
train = train.reset_index(drop = True)
test = test.reset_index(drop = True)
val = val.reset_index(drop = True)

y_train = train['churn'].values
y_val = val['churn'].values
y_test = test['churn'].values

del train['churn']
del test['churn']
del val['churn']

In [66]:
len(y_train), len(y_val), len(y_test)

(4225, 1409, 1409)

In [55]:
full_train.isnull().sum()

customerid          0
gender              0
seniorcitizen       0
partner             0
dependents          0
tenure              0
phoneservice        0
multiplelines       0
internetservice     0
onlinesecurity      0
onlinebackup        0
deviceprotection    0
techsupport         0
streamingtv         0
streamingmovies     0
contract            0
paperlessbilling    0
paymentmethod       0
monthlycharges      0
totalcharges        0
churn               0
dtype: int64

In [67]:
full_train.churn.value_counts(normalize = True)

0    0.730032
1    0.269968
Name: churn, dtype: float64

In [70]:
full_train.columns

Index(['customerid', 'gender', 'seniorcitizen', 'partner', 'dependents',
       'tenure', 'phoneservice', 'multiplelines', 'internetservice',
       'onlinesecurity', 'onlinebackup', 'deviceprotection', 'techsupport',
       'streamingtv', 'streamingmovies', 'contract', 'paperlessbilling',
       'paymentmethod', 'monthlycharges', 'totalcharges', 'churn'],
      dtype='object')

In [72]:
categorical =['gender', 'seniorcitizen', 'partner', 'dependents',
       'phoneservice', 'multiplelines', 'internetservice',
       'onlinesecurity', 'onlinebackup', 'deviceprotection', 'techsupport',
       'streamingtv', 'streamingmovies', 'contract', 'paperlessbilling',
       'paymentmethod']

In [73]:
numerical  = ['tenure', 'totalcharges', 'monthlycharges']

In [86]:
from IPython.display import display
for c in categorical:
  print(c)
  df_group = full_train.groupby(c).churn.agg(['mean', 'count'])
  df_group['diff'] = df_group['mean'] - full_train['churn'].mean()
  df_group['risk'] = df_group['mean'] / full_train['churn'].mean()
  display(df_group)
  print()
  print()

gender


,mean,count,diff,risk
gender,,,,
female,0.276824,2796,0.006856,1.025396
male,0.263214,2838,-0.006755,0.974980




seniorcitizen


,mean,count,diff,risk
seniorcitizen,,,,
0,0.242270,4722,-0.027698,0.897403
1,0.413377,912,0.143409,1.531208




partner


...

,mean,count,diff,risk
contract,,,,
month-to-month,0.431701,3104,0.161733,1.599082
one_year,0.120573,1186,-0.149395,0.446621
two_year,0.028274,1344,-0.241694,0.104730




paperlessbilling


,mean,count,diff,risk
paperlessbilling,,,,
no,0.172071,2313,-0.097897,0.637375
yes,0.338151,3321,0.068183,1.252560




paymentmethod


,mean,count,diff,risk
paymentmethod,,,,
bank_transfer_(automatic),0.168171,1219,-0.101797,0.622928
credit_card_(automatic),0.164339,1217,-0.105630,0.608733
electronic_check,0.455890,1893,0.185922,1.688682
mailed_check,0.193870,1305,-0.076098,0.718121


In [87]:
from sklearn.metrics import mutual_info_score

def mutual_info_scroe_fun(series):
  
  return mutual_info_score(series, full_train.churn)

In [89]:
muti = full_train[categorical].apply(mutual_info_scroe_fun)
muti.sort_values(ascending = False)

contract            0.098320
onlinesecurity      0.063085
techsupport         0.061032
internetservice     0.055868
onlinebackup        0.046923
deviceprotection    0.043453
paymentmethod       0.043210
streamingtv         0.031853
streamingmovies     0.031581
paperlessbilling    0.017589
dependents          0.012346
partner             0.009968
seniorcitizen       0.009410
multiplelines       0.000857
phoneservice        0.000229
gender              0.000117
dtype: float64

In [90]:
full_train[numerical].corrwith(full_train.churn).abs()

tenure            0.351885
totalcharges      0.196353
monthlycharges    0.196805
dtype: float64

In [91]:
from sklearn.feature_extraction import DictVectorizer

dv = DictVectorizer(sparse = False)

train_dict = train[categorical + numerical].to_dict(orient = 'records')
X_train = dv.fit_transform(train_dict)

val_dict = val[categorical + numerical].to_dict(orient='records')
X_val = dv.transform(val_dict)

In [92]:
from sklearn.linear_model import LogisticRegression

model = LogisticRegression(solver='lbfgs')
model.fit(X_train, y_train)

LogisticRegression()

In [93]:
model.intercept_[0]

-0.10903461122930498

In [98]:
model.coef_[0].round(3)

array([ 0.475, -0.175, -0.408, -0.03 , -0.078,  0.063, -0.089, -0.081,
       -0.034, -0.073, -0.335,  0.317, -0.089,  0.004, -0.258,  0.141,
        0.009,  0.063, -0.089, -0.081,  0.266, -0.089, -0.284, -0.231,
        0.124, -0.166,  0.058, -0.087, -0.032,  0.07 , -0.059,  0.141,
       -0.249,  0.215, -0.12 , -0.089,  0.102, -0.071, -0.089,  0.052,
        0.213, -0.089, -0.232, -0.07 ,  0.   ])

In [105]:
y_pred = model.predict_proba(X_val)[:, 1]

In [107]:
churn_decision = (y_pred >= 0.5)

In [108]:
(y_val == churn_decision).mean()

0.8034066713981547

In [109]:
dict(zip(dv.get_feature_names(), model.coef_[0].round(3)))

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


{'contract=month-to-month': 0.475,
 'contract=one_year': -0.175,
 'contract=two_year': -0.408,
 'dependents=no': -0.03,
 'dependents=yes': -0.078,
 'deviceprotection=no': 0.063,
 'deviceprotection=no_internet_service': -0.089,
 'deviceprotection=yes': -0.081,
 'gender=female': -0.034,
 'gender=male': -0.073,
 'internetservice=dsl': -0.335,
 'internetservice=fiber_optic': 0.317,
 'internetservice=no': -0.089,
 'monthlycharges': 0.004,
 'multiplelines=no': -0.258,
 'multiplelines=no_phone_service': 0.141,
 'multiplelines=yes': 0.009,
 'onlinebackup=no': 0.063,
 'onlinebackup=no_internet_service': -0.089,
 'onlinebackup=yes': -0.081,
 'onlinesecurity=no': 0.266,
 'onlinesecurity=no_internet_service': -0.089,
 'onlinesecurity=yes': -0.284,
 'paperlessbilling=no': -0.231,
 'paperlessbilling=yes': 0.124,
 'partner=no': -0.166,
 'partner=yes': 0.058,
 'paymentmethod=bank_transfer_(automatic)': -0.087,
 'paymentmethod=credit_card_(automatic)': -0.032,
 'paymentmethod=electronic_check': 0.07,
 

In [110]:
full_train_dict = full_train[categorical + numerical].to_dict(orient = 'records')
X_full_train = dv.fit_transform(full_train_dict)

y_full_train = full_train.churn.values

full_model = LogisticRegression(solver = 'lbfgs')
full_model.fit(X_full_train, y_full_train)

dict_test = test[categorical + numerical].to_dict(orient = 'records')
X_test = dv.transform(dict_test)

y_full_pred = full_model.predict_proba(X_test)[:, 1]

churn_decision = (y_full_pred >= 0.5)

(y_test == churn_decision).mean()

0.815471965933286

In [111]:
customer = dict_test[470]
customer

{'contract': 'one_year',
 'dependents': 'no',
 'deviceprotection': 'yes',
 'gender': 'female',
 'internetservice': 'fiber_optic',
 'monthlycharges': 93.5,
 'multiplelines': 'no',
 'onlinebackup': 'yes',
 'onlinesecurity': 'yes',
 'paperlessbilling': 'yes',
 'partner': 'yes',
 'paymentmethod': 'electronic_check',
 'phoneservice': 'yes',
 'seniorcitizen': 0,
 'streamingmovies': 'no',
 'streamingtv': 'yes',
 'techsupport': 'no',
 'tenure': 51,
 'totalcharges': 4619.55}

In [112]:
customer_test = dv.transform(customer)

In [114]:
full_model.predict_proba(customer_test)[0,1]

0.14265834297078928

In [115]:
y_test[470]

0

In [122]:
for i in range(100):
  print(test.customerid[i])
  customer_id = dict_test[i]
  customer_id_test = dv.transform([customer_id])
  predicition = full_model.predict_proba(customer_id_test)[0,1]
  print(y_test[i])
  print(predicition)
  #dis = pd.DataFrame()
  #dis['predicition'] = predicition
  #dis['Real'] = y_test[i]
  #display(dis)
  print()
  print()


8879-zkjof
0
0.06224295541712798


0201-mibol
0
0.17473875283664028


1600-dilpe
0
0.3702670092568016


8601-qacrs
1
0.5093610645419581


7919-zodzz
0
0.23367939545286417


7964-zrkkg
0
0.13215882880602686


1357-bijki
0
0.3684316428045189


8720-rqsbj
0
0.5299229687344803


4682-blbuc
0
0.07547539724328994


8930-xotdp
0
0.2142348101436834


0111-klbqg
0
0.4056810977859304


2898-mrkpi
0
0.16754607127574322


6168-wfvvf
1
0.7587765201319641


0178-szbho
0
0.2669811538946235


5469-nujur
0
0.0694870792181527


4836-wnfno
0
0.0037273494473083224


0064-yijgf
0
0.4342315512831051


7682-azndk
0
0.3780455046783127


5153-rthkf
0
0.46376110859225256


5973-ejgdp
0
0.21239477301940082


5868-ytyks
0
0.23186665477392188


8623-tmrby
1
0.288967920171471


4163-kiuhy
0
0.09931126866716863


6993-ycobk
0
0.14196178835339893


2040-xbavj
0
0.04639776292795211


9470-rtwdv
0
0.1722992761840098


3197-nnynb
0
0.12676012741672232


7817-omjna
0
0.07472589490740716


6131-iunxn
0
0.06594996484262015